# **Object Detection - Models Training**

---

**Mateus Aleixo - 124256**

---

In [6]:
from collections import Counter
from glob import glob
import os
import random
import shutil
import torch
from ultralytics import YOLO

In [7]:
def get_image_label_pairs(
    image_root, annotation_root, image_ext="jpg", label_ext="txt"
):
    pairs = []
    image_files = glob(os.path.join(image_root, "**", f"*.{image_ext}"), recursive=True)

    for img in image_files:
        rel_path = os.path.relpath(img, image_root)
        label = os.path.join(
            annotation_root, os.path.splitext(rel_path)[0] + f".{label_ext}"
        )

        if os.path.exists(label):
            pairs.append((img, label))
        else:
            print(f"Warning: Label file not found for image {img}")

    return pairs

In [8]:
image_root = "../data/augmented_data/frames"
annotation_root = "../data/annotations"
pairs = get_image_label_pairs(image_root, annotation_root)
print(f"Found {len(pairs)} image-label pairs.")

Found 3864 image-label pairs.


In [9]:
empty_files = []

for root, _, files in os.walk(annotation_root):
    for file in files:
        file_path = os.path.join(root, file)

        if os.stat(file_path).st_size == 0:
            empty_files.append(file_path)

print(f"Found {len(empty_files)} empty label files.")

Found 0 empty label files.


In [10]:
label_counts = Counter()

for root, _, files in os.walk(annotation_root):
    for file in files:
        with open(os.path.join(root, file), "r") as f:
            lines = f.readlines()

            for line in lines:
                class_id = line.split()[0]
                label_counts[class_id] += 1

label_counts

Counter({'1': 7728, '2': 7728, '3': 7728, '0': 7726, '4': 7694})

In [11]:
# Paths
image_dir = "../data/augmented_data/frames"
label_dir = "../data/annotations"
output_dir = "../data/split_data"

# Create output folders
for split in ["train", "val"]:
    os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

# Get all images
all_images = []
for root, _, files in os.walk(image_dir):
    for file in files:
        if file.endswith(".jpg") or file.endswith(".png"):
            all_images.append(os.path.join(root, file))

# Shuffle and split
random.shuffle(all_images)
train_ratio = 0.8
train_count = int(len(all_images) * train_ratio)

train_images = all_images[:train_count]
val_images = all_images[train_count:]


# Move files
def move_files(image_list, split):
    for img_path in image_list:
        # Get label path
        rel_path = os.path.relpath(img_path, image_dir)
        label_path = os.path.join(label_dir, os.path.splitext(rel_path)[0] + ".txt")

        # Define new paths
        new_img_path = os.path.join(
            output_dir, "images", split, os.path.basename(img_path)
        )
        new_label_path = os.path.join(
            output_dir, "labels", split, os.path.basename(label_path)
        )

        # Copy files
        shutil.copy(img_path, new_img_path)
        if os.path.exists(label_path):
            shutil.copy(label_path, new_label_path)


# Move train & val files
move_files(train_images, "train")
move_files(val_images, "val")

print(f"Dataset split completed: {len(train_images)} train, {len(val_images)} val")

Dataset split completed: 3091 train, 773 val


In [13]:
model = YOLO(verbose=False)

results = model.train(
    data="../data/dataset.yaml",
    epochs=22,
    batch=8,
    imgsz=416,
    device="cuda" if torch.cuda.is_available() else "cpu",
    optimizer="AdamW",
    lr0=0.001,
    lrf=0.01,
    weight_decay=0.0005,
    patience=5,
    degrees=10,
    translate=0.2,
    scale=0.5,
    shear=2.0,
    perspective=0.001,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.2,
)

New https://pypi.org/project/ultralytics/8.3.101 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.40  Python-3.12.9 torch-2.6.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=../data/dataset.yaml, epochs=22, time=None, patience=5, batch=8, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fra

train: Scanning C:\Users\mateu\Downloads\UA\Cadeiras\Seminário\Object_Detection\data\split_data\labels\train... 2203 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2203/2203 [00:03<00:00, 675.02it/s]


train: New cache created: C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\data\split_data\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


c:\Users\mateu\Downloads\UA\Cadeiras\Seminário\Object_Detection\.venv\Lib\site-packages\ultralytics\data\augment.py:1850: UserWarning: Argument(s) 'quality_lower' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=75, p=0.0),
val: Scanning C:\Users\mateu\Downloads\UA\Cadeiras\Seminário\Object_Detection\data\split_data\labels\val... 1179 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1179/1179 [00:01<00:00, 820.09it/s]


val: New cache created: C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\data\split_data\labels\val.cache
Plotting labels to c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train
Starting training for 22 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      22/22         0G     0.9554     0.4668     0.8111         28        416: 100%|██████████| 276/276 [02:53<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 74/74 [00:32<00:00,  2.29it/s]

                   all       1179      11782      0.993      0.992      0.993      0.769



22 epochs completed in 1.312 hours.
Optimizer stripped from c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train\weights\last.pt, 5.5MB
Optimizer stripped from c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train\weights\best.pt, 5.5MB

Validating c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train\weights\best.pt...
Ultralytics 8.3.40  Python-3.12.9 torch-2.6.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)
YOLO11n summary (fused): 238 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 74/74 [00:28<00:00,  2.58it/s]


                   all       1179      11782      0.992      0.992      0.993      0.788
                  blue       1179       2358      0.995      0.996      0.992      0.807
                 green       1179       2358       0.99      0.982      0.993      0.752
                orange       1179       2358      0.994      0.994      0.993      0.797
                   red       1179       2358       0.99      0.996      0.993      0.801
                yellow       1175       2350       0.99      0.993      0.992       0.78
Speed: 0.4ms preprocess, 17.5ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train
